In [2]:
!pip install kaggle --upgrade



In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d thienkhonghoc/affectnet -p /content

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown


In [5]:
!unzip -q /content/affectnet.zip -d /content/affectnet > /dev/null 2>&1

In [6]:
!pip install torch torchvision timm matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from collections import Counter
from PIL import Image
import os

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Transforms (Progressive Resize + Random Erasing) ===
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB") if isinstance(img, Image.Image) else img),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomErasing(p=0.4, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === Load Datasets ===
train_path = "/content/affectnet/AffectNet/train"
val_path = "/content/affectnet/AffectNet/val"

train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
val_dataset = datasets.ImageFolder(root=val_path, transform=transform)

# Remove corrupts
def filter_images(dataset):
    dataset.samples = [(p, l) for p, l in dataset.samples if Image.open(p).convert("RGB")]

filter_images(train_dataset)
filter_images(val_dataset)

# Class weights
class_counts = Counter(train_dataset.targets)
total = sum(class_counts.values())
weights = torch.tensor([total / class_counts[i] for i in range(len(class_counts))], dtype=torch.float).to(device)

# Loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# === Load ConvNeXt-Large Model ===
model = models.convnext_large(weights=models.ConvNeXt_Large_Weights.IMAGENET1K_V1)
model.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.LayerNorm(model.classifier[2].in_features),
    nn.Dropout(0.5),
    nn.Linear(model.classifier[2].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 8)
)

# Load SWA weights
checkpoint = torch.load("/content/affectnet_convnext_large_swa.pt", map_location=device)
cleaned = {k.replace("_orig_mod.", ""): v for k, v in checkpoint.items()}
model.load_state_dict(cleaned, strict=False)
model = model.to(device)

# Criterion, Optimizer, LR Scheduler
criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.05)
optimizer = optim.AdamW(model.parameters(), lr=5e-6, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-7)
scaler = GradScaler()

# === Fine-tuning ===
print("Loaded SWA checkpoint. Fine-tuning from Epoch 61 → 65...\n")

for epoch in range(61, 66):
    model.train()
    correct, total, train_loss = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # === Validation ===
    model.eval()
    correct_val, total_val, val_loss = 0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            correct_val += (outputs.argmax(1) == labels).sum().item()
            total_val += labels.size(0)

    val_acc = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)
    scheduler.step()

    print(f"Epoch [{epoch}/65] | Train Acc: {train_acc:.2f}% | Train Loss: {avg_train_loss:.4f} | "
          f"Val Acc: {val_acc:.2f}% | Val Loss: {avg_val_loss:.4f}")

    if epoch % 1 == 0:
        save_path = f"affectnet_convnext_large_epoch{epoch}.pt"
        torch.save(model.state_dict(), save_path)
        print(f"Model saved: {save_path}")

# Final save
torch.save(model.state_dict(), "affectnet_convnext_large_final.pt")
print("\n Fine-tuning complete. Final model saved.")


Loaded SWA checkpoint. Fine-tuning from Epoch 61 → 65...



<ipython-input-1-b8db257a702c>:76: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-1-b8db257a702c>:89: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-1-b8db257a702c>:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [61/65] | Train Acc: 78.95% | Train Loss: 0.7878 | Val Acc: 56.62% | Val Loss: 1.5856
Model saved: affectnet_convnext_large_epoch61.pt
Epoch [62/65] | Train Acc: 79.93% | Train Loss: 0.7680 | Val Acc: 57.50% | Val Loss: 1.5658
Model saved: affectnet_convnext_large_epoch62.pt
Epoch [63/65] | Train Acc: 80.44% | Train Loss: 0.7587 | Val Acc: 59.25% | Val Loss: 1.5276
Model saved: affectnet_convnext_large_epoch63.pt
Epoch [64/65] | Train Acc: 81.22% | Train Loss: 0.7379 | Val Acc: 59.62% | Val Loss: 1.5408
Model saved: affectnet_convnext_large_epoch64.pt
Epoch [65/65] | Train Acc: 81.48% | Train Loss: 0.7336 | Val Acc: 57.25% | Val Loss: 1.5763
Model saved: affectnet_convnext_large_epoch65.pt

 Fine-tuning complete. Final model saved.
